In [1]:
!

# Evaluation of mirkatai's hability to retrive the correct information

This will be the evaluation on the correctness and time of the SQL module of th AI agente miRKAtAI vs the expected output.

File sql_eval_question_answer.tsv contains the questions, the query to get the rigth answer and the Answer. 
File dificulty.tsv has the question, if the answer has a unique word or numbes as answer (Unique answer) and a Dificulty column that goes from 1 to 4. Being 1 the basic instructions



In [2]:
base_path = '/home/karen/Documents/GitHub/MirKatAI/evaluation/data'
questions_file = f'{base_path}/sql_eval_question_answer.tsv'
difficulty_file = f'{base_path}/dificulty.tsv'


In [3]:
%matplotlib inline

import pandas as pd
import numpy as np
import app.mirkat.sql_functions as sql_functions

import os
import app.nodes as nodes

import json
from langchain_core.messages import (
    AIMessage,
    SystemMessage
)
import time

/home/karen/Documents/GitHub/MirKatAI/.venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
2025-10-14 20:28:59,269 - INFO - package: mysql.connector.plugins
2025-10-14 20:28:59,280 - INFO - plugin_name: caching_sha2_password
2025-10-14 20:28:59,280 - INFO - AUTHENTICATION_PLUGIN_CLASS: MySQLCachingSHA2PasswordAuthPlugin


In [4]:
import concurrent.futures


In [5]:
# Collect questions
questions_df = pd.read_csv(questions_file, sep='\t', index_col=0)
questions_df = questions_df.dropna(subset=['Question', 'Query', 'Answer'])
questions_df = questions_df.drop_duplicates(subset=['Question', 'Query', 'Answer'])
questions_df = questions_df.reset_index(drop=True)

# Collect dificulty
difficulty_df = pd.read_csv(difficulty_file, sep='\t', index_col=0)
difficulty_df = difficulty_df.dropna(subset=['Question', 'Unique answer', 'Dificulty'])
difficulty_df = difficulty_df.drop_duplicates(subset=['Question', 'Unique answer', 'Dificulty'])
difficulty_df = difficulty_df.reset_index(drop=True)
# Merge questions and difficulty
questions_df = pd.merge(questions_df, difficulty_df, on='Question', how='left')
questions_df = questions_df.dropna(subset=['Dificulty'])
questions_df = questions_df.reset_index(drop=True)

questions_df.head(2)

,Question,Query,Answer,Unique answer,Dificulty
0,How many unique mature microRNAs are in the da...,SELECT COUNT(DISTINCT auto_mature) FROM mirna_...,49155,Y,1
1,List all known species (organisms) for which m...,SELECT DISTINCT name FROM mirna_species;,+---------------------------------------------...,N,1


## Manual SQL queries

In [6]:
from dotenv import load_dotenv


from app.mirkat.sql_functions import (
    MySqlConnection,
    DBTools
)

mirkat_path = "/home/karen/Documents/GitHub/MirKatAI/"


# Load .env file
load_dotenv()
# Get the API key


pwd = os.getcwd()


mirkat_columns_desctiption = pd.read_csv(mirkat_path+'tables/mirkat_tables_columns_descriptions.csv')
mirkat_tables_desctiption = pd.read_csv(mirkat_path+'tables/mirkat_tables_descriptions.csv')


In [7]:
### SQL connection


config = {
    'user': os.getenv('MIRKAT_USER'),
    'password': os.getenv('MIRKAT_PASSWORD'),
    'host': os.getenv('MIRKAT_HOST'),
    'database': os.getenv('MIRKAT_DATABASE'),
    'raise_on_warnings': True
}

mysql_connection = sql_functions.MySqlConnection(config)
db_conn = mysql_connection.connect_sql()

db_tools = DBTools(db_conn, mirkat_tables_desctiption, mirkat_columns_desctiption)
def reset_sql_conf():
    # This functions just restart the db connections
    global mysql_connection, db_conn, db_tools

    mysql_connection = sql_functions.MySqlConnection(config)
    db_conn = mysql_connection.connect_sql()

    db_tools = DBTools(db_conn, mirkat_tables_desctiption, mirkat_columns_desctiption)




## Agent based sql queries

In [8]:

_,_,_,sql_node,_ = nodes.create_nodes()


In [9]:
import multiprocessing
# add logs
import logging
# Logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('mirkat_sql_evaluation.log'),
        logging.StreamHandler()
    ],
    encoding='utf-8'  # Ensure log messages are UTF-8 encoded
)

def run_with_timeout(func, *args, timeout=30, **kwargs):
    def wrapper(q, *args, **kwargs):
        try:
            result = func(*args, **kwargs)
            q.put(("ok", result))
        except Exception as e:
            q.put(("error", e))

    q = multiprocessing.Queue()
    p = multiprocessing.Process(target=wrapper, args=(q, *args), kwargs=kwargs)
    p.start()
    p.join(timeout)

    if p.is_alive():
        print("TimeoutError at run_with_timeout")
        p.terminate()
        p.join()
        logging.error("Operation time out")
        raise TimeoutError("Operation timed out")

    status, value = q.get()
    if status == "error":
        logging.error(f"Error occurred in run_with_timeout: {value}")
        raise value
    print(f"In run_with_timeout, response is type {type(value)} and value is {value}")
    logging.info(f"In run_with_timeout, response is type {type(value)} and value is {value}")
    return value


def sql_run_model(query: str):
    """
    This will run the query using the AI model
    """
    _,_,_,sql_node,_ = nodes.create_nodes()
    logging.info(f"Running sql_run_model with query: {query}")
    messages = f"***SQL*** {query}"
    ai_message = AIMessage(content=messages)
    start_time = pd.Timestamp.now()
    # add response timeout if the call for sql_node takes more than 30 sec
    try:
        result = sql_node.run_model(ai_message)
    except TimeoutError:
        logging.warning(f"Timeout question {query}")
        result = AIMessage(content="Timeout")
    except Exception as e:
        print(f"Error question {query}: {e}")
        logging.error(f"Error question {query}: {e}")
        result = AIMessage(content=f"Error: {e}")
        
    end_time = pd.Timestamp.now()
    time_taken = (end_time - start_time).total_seconds()
    if type(result.text) == str:
        result_content = result.text
    else:
        result_content = result.text()
    return result_content, time_taken
def sql_run(query: str):
    """
    This will run the query using the MySQL connection
    """
    start_time = pd.Timestamp.now()
    try:
        result = db_tools.execute_query(query, 'test_query')
    except TimeoutError:
        result = "Timeout"
        print(f"Timeout question {query}")
    end_time = pd.Timestamp.now()
    time_taken = (end_time - start_time).total_seconds()
    return result, time_taken



In [10]:
def evaluate_unique_answer(unique_answer_question, asnwers_log = '/home/karen/Documents/GitHub/MirKatAI/evaluation/results/unique.txt', n=1, answer_registry={}):
    # for every question with Unique answer = Y, run the agent and evaluate if the answer contains the unique answer. Register the time it took
    n_order = 0
    for question_row in unique_answer_question.iterrows():
        
        print (question_row, "type", type(question_row))
        question = question_row[1]['Question']
        print('Question:', question)
        true_answer = question_row[1]['Answer']
        # run agent
        n_order = n_order + 1
        answer, response_time = sql_run_model(question)
        # Check if the agent's response contains the unique answer
        print(answer, response_time)
        correct_flag = true_answer in answer
        #append answer to answers_log
        with open(asnwers_log, 'a') as log_file:
            log_file.write(f"{time.time()} Q{n_order}: {question} - try {n} - Answer: {answer} - Rwesponse time: {response_time} \n")
        if question in answer_registry:
            answer_registry[question]['Response Time'].append(response_time)
            answer_registry[question]['Correct'].append(correct_flag)
            answer_registry[question]['Expected'] = true_answer
            answer_registry[question]['Answers'].append(answer)
            answer_registry[question]['Question_n'].append(n_order)
        else:
            answer_registry[question] = {
                'Response Time': [response_time],
                'Correct': [correct_flag],
                'Expected': true_answer,
                'Answers': [answer],
                'Question_n': [n_order]
            }
        time.sleep(2)
            

    return answer_registry


In [11]:
unique_answers_df = questions_df[questions_df['Unique answer']=='Y']

In [28]:
answer_registry ={}
sql_node = nodes.sql_node
long_rest_interval = 10  # Number of iterations after which to take a long rest
long_rest_duration = 300  # Duration of the long rest in seconds (5 minutes)
current_iteration = 0
for i in range(20):

    # order randomly the questions
    #unique_answers_random_df = unique_answers_df.sample(frac=1).reset_index(drop=True)
    # get only question 'In which tissue is the miRNA 'hsa-miR-143-3p' most highly expressed ' or  'In which tissue is the miRNA 'hsa-miR-143-3p' most highly expressed'
    unique_answers_random_df = unique_answers_df[unique_answers_df['Question'].isin(["In which tissue is the miRNA 'hsa-miR-143-3p' most highly expressed ", "In which tissue is the miRNA 'hsa-miR-143-5p' most highly expressed"])]
    display(unique_answers_random_df)
    answer_registry =evaluate_unique_answer(unique_answers_random_df, asnwers_log = '/home/karen/Documents/GitHub/MirKatAI/evaluation/results/unique.txt', n=i, answer_registry=answer_registry)

    current_iteration += 1
    if current_iteration % long_rest_interval == 0:
        print(f"Taking a long rest after {current_iteration} iterations.")
        time.sleep(long_rest_duration)
        current_iteration = 0  
    time.sleep(5)


,Question,Query,Answer,Unique answer,Dificulty


KeyboardInterrupt: 

In [26]:
unique_answers_df

,Question,Query,Answer,Unique answer,Dificulty
0,How many unique mature microRNAs are in the da...,SELECT COUNT(DISTINCT auto_mature) FROM mirna_...,49155,Y,1
2,Which species from the database is marked with...,Select name from mirna_species group by name h...,Merkel cell polyomavirus,Y,2
3,How many miRNA target interactions are recorde...,SELECT count(*) FROM gene_mirna WHERE mirna_ma...,1492,Y,1
7,What is the seed sequence for 'hsa-miR-9-5p'?,Select seed from mirna_seeds where auto_mature...,CUUUGGU,Y,1
8,How many interactions are experimentally valid...,SELECT count(*) FROM gene_mirna WHERE source =...,7694,Y,1
9,What is the probability for the predicted inte...,select * from gene_mirna where mirna_mature = ...,0,Y,1
20,How many unique miRNAs target the gene 'GRM5' ...,select count(distinct mirna_mature) from gene_...,307,Y,1
21,How many unique miRNAs target the gene 'FEZ2' ...,select count(distinct mirna_mature) from gene_...,75,Y,1
22,How many unique miRNAs target the gene 'STUM' ...,select count(distinct mirna_mature) from gene_...,278,Y,1
23,How many mirnas that target FEZ2 ( including i...,SELECT COUNT(DISTINCT mt.mirna) AS num_mir...,2,Y,2


In [16]:
pd.DataFrame(answer_registry)

,How many unique miRNAs target the gene 'GRM5' including its reference sequences ?,How many mirnas that target FEZ2 ( including its reference sequences) are also expressed in muscle with a organ_TSI > 0.5,How many unique mature microRNAs are in the database?,Which species from the database is marked with two or more organisms,How many gene-miRNA interactions are recorded that originate from the database 'miRDB'?,What is the minimum interaction score recorded for any miRNA targeting the gene 'VEGFA' including its reference sequences ?,How many miRNA target interactions are recorded for the miRNA 'hsa-miR-143-3p'?,What is the seed sequence for 'hsa-miR-9-5p'?,What is the probability for the predicted interaction between 'hsa-miR-22-3p' and the gene 'SIRT1' including its reference sequences (refseq)?,How many unique miRNAs target the gene 'STUM' including its reference sequences ?,How many mirnas that target STUM ( including its reference sequences ) are also expressed in muscle with a organ_TSI > 0.5,How many interactions are experimentally validated (source is 'miRTarBase')?,How many unique miRNAs target the gene 'FEZ2' including its reference sequences ?,How many unique seed sequences are associated with human miRNAs?,In which tissue is the miRNA 'hsa-miR-143-3p' most highly expressed,How many gene-miRNA interactions are recorded that originate from the database 'miRDB' from unique gene symbols?
Response Time,"[12.886559, 12.086426, 12.096433, 12.007489, 1...","[2.491516, 2.460368, 2.794193, 2.696787, 3.212...","[1.557451, 1.7662499999999999, 1.613115, 1.702...","[0.739929, 0.679385, 0.743183, 0.777748, 0.636...","[9.471009, 9.360354, 9.547693, 9.8698529999999...","[13.443455, 13.52783, 13.64605, 14.669205, 13....","[1.665988, 1.816568, 2.095855, 1.850717, 1.875...","[1.66527, 1.641788, 1.717432, 1.868747, 1.7157...","[5.05319, 5.068398, 5.562333, 5.426047, 5.3411...","[12.114549, 0.64081, 0.846683, 0.712854, 0.718...","[2.510014, 2.6769439999999998, 2.582528, 2.690...","[9.017892, 8.837927, 9.067635, 9.040962, 9.135...","[11.904678, 18.460946, 12.214048, 11.832598, 1...","[1.870697, 1.851346, 2.075484, 2.065714, 2.020...","[3.929695, 4.054056, 4.437605, 4.385544, 4.184...","[14.569184, 14.568149, 14.065654, 8.161417, 14..."
Correct,"[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal..."
Expected,307,2,49155,Merkel cell polyomavirus,6831595,50,1492,CUUUGGU,0,278,4,7694,75,699,bladder,17352
Answers,[There are 97 unique miRNAs that target the ge...,[There are 2 microRNAs that target FEZ2 (inclu...,[There are 49155 unique mature microRNAs in th...,"[I am sorry, I cannot fulfill this request. Th...","[There are 6,831,595 gene-miRNA interactions r...",[The minimum probability for miRNA targeting V...,[There are 1492 miRNA target interactions reco...,[The seed sequence for hsa-miR-9-5p is CUUUGGU...,[The query returned multiple probabilities. It...,[There are 95 unique miRNAs that target the ge...,[There are 2 microRNAs that target STIM1 (incl...,[There are 7694 interactions from miRTarBase.\...,[There are 75 unique miRNAs that target the ge...,[There are 2027 unique seed sequences associat...,[The miRNA 'hsa-miR-143-3p' is most highly exp...,[There are 147959 unique gene

In [17]:
answer_registry[list(answer_registry.keys())[0]].keys()

dict_keys(['Response Time', 'Correct', 'Expected', 'Answers', 'Question_n'])

In [ ]:
# save first_answer_registry to json
with open('/home/karen/Documents/GitHub/MirKatAI/evaluation/results/answer_registry_19.json', 'a') as f:
    json.dump(answer_registry, f)

In [19]:
# evaluate with direct sql calls



def get_answer_time_sql(unique_answer_questions, answers_log = "/home/karen/Documents/GitHub/MirKatAI/evaluation/results/sql_unique.txt", n=1, answer_registry={}):

    for question_row in unique_answer_questions.iterrows():
        question = question_row[1]['Question']
        query = question_row[1]['Query']
        answer, response_time = sql_run(query=query)
        print(answer, response_time)
        #append answer to answers_log
        with open(answers_log, 'a') as log_file:
            log_file.write(f"{question} - try {n} - Answer: {answer}")
        if question in answer_registry:
            answer_registry[question]['Response Time'].append(response_time)
        else:
            answer_registry[question] = {
                'Response Time': [response_time]
            }
        time.sleep(1)
    return answer_registry
        

In [20]:
# Run directly with SQL calls

answer_registry_sql ={}
for i in range(20):
    answer_registry_sql = get_answer_time_sql(unique_answer_questions=unique_answers_df, n=i, answer_registry=answer_registry_sql)
    time.sleep(2)



2025-10-15 00:09:20,768 - INFO -  - DB CALL: execute_query(SELECT COUNT(DISTINCT auto_mature) FROM mirna_mature;)


 - DB CALL: execute_query(SELECT COUNT(DISTINCT auto_mature) FROM mirna_mature;)
Results from SQL are: [(49168,)]
[(49168,)] 0.260816


2025-10-15 00:09:22,039 - INFO -  - DB CALL: execute_query(Select name from mirna_species group by name having count(distinct organism) >1;)


 - DB CALL: execute_query(Select name from mirna_species group by name having count(distinct organism) >1;)
Results from SQL are: [('Merkel cell polyomavirus',)]
[('Merkel cell polyomavirus',)] 0.043774


2025-10-15 00:09:23,084 - INFO -  - DB CALL: execute_query(SELECT count(*) FROM gene_mirna WHERE mirna_mature = 'hsa-miR-143-3p')


...

 - DB CALL: execute_query(select min(gm.probability)  from gene_mirna gm where gm.mrna in (select refseq_mrna from gene_names where gene_symbol = 'VEGFA') or gm.mrna = 'VEGFA' and gm.source = 'miRDB'; )
Results from SQL are: []
[] 0.025758


2025-10-15 00:33:21,210 - INFO -  - DB CALL: execute_query(SELECT COUNT(*) FROM gene_mirna WHERE source = 'miRDB';)


 - DB CALL: execute_query(SELECT COUNT(*) FROM gene_mirna WHERE source = 'miRDB';)
Results from SQL are: [(6831595,)]
[(6831595,)] 14.623516


2025-10-15 00:33:36,835 - INFO -  - DB CALL: execute_query(SELECT COUNT(DISTINCT gn.gene_symbol) AS unique_gene_symbols_with_mirdb_interactions FROM gene_mirna gm JOIN gene_names gn ON gm.mrna = gn.RefSeq_mRNA WHERE gm.source = 'miRDB';)


 - DB CALL: execute_query(SELECT COUNT(DISTINCT gn.gene_symbol) AS unique_gene_symbols_with_mirdb_interactions FROM gene_mirna gm JOIN gene_names gn ON gm.mrna = gn.RefSeq_mRNA WHERE gm.source = 'miRDB';)
Results from SQL are: [(17352,)]
[(17352,)] 5.756081


In [21]:
answer_registry_sql
# save first_answer_registry to json
with open('/home/karen/Documents/GitHub/MirKatAI/evaluation/results/answer_registry_sql_7.json', 'w') as f:
    json.dump(answer_registry_sql, f)

In [ ]:
db_tools.execute_query(sql="Select name from mirna_species group by name having count(distinct organism) >1;", query_name="query")

2025-10-13 21:00:40,261 - INFO -  - DB CALL: execute_query(Select name from mirna_species group by name having count(distinct organism) >1;)


 - DB CALL: execute_query(Select name from mirna_species group by name having count(distinct organism) >1;)
Results from SQL are: [('Merkel cell polyomavirus',)]


[('Merkel cell polyomavirus',)]

In [ ]:
unique_answers_df['Question'].values

array(['How many unique mature microRNAs are in the database?',
       'Which species from the database is marked with two or more organisms',
       "How many miRNA target interactions are recorded for the miRNA 'hsa-miR-143-3p'?",
       "What is the seed sequence for 'hsa-miR-9-5p'?",
       "How many interactions are experimentally validated (source is 'miRTarBase')?",
       "What is the probability for the predicted interaction between 'hsa-miR-22-3p' and the gene 'SIRT1' including its reference sequences (refseq)?",
       "How many unique miRNAs target the gene 'GRM5'  including its reference sequences ?",
       "How many unique miRNAs target the gene 'FEZ2'  including its reference sequences ?",
       "How many unique miRNAs target the gene 'STUM'  including its reference sequences ?",
       'How many mirnas that target FEZ2 ( including its reference sequences) are also expressed in muscle with a organ_TSI > 0.5',
       'How many mirnas that target STUM ( including its ref